In [ ]:
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu

In [ ]:
import malaya
import re
from malaya.texts._text_functions import split_into_sentences
from malaya.texts import _regex
import numpy as np
import itertools

tokenizer = malaya.preprocessing._tokenizer
splitter = split_into_sentences

In [ ]:
def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def preprocessing(w):
    if is_number_regex(w):
        return '<NUM>'
    elif re.match(_regex._money, w):
        return '<MONEY>'
    elif re.match(_regex._date, w):
        return '<DATE>'
    elif re.match(_regex._expressions['email'], w):
        return '<EMAIL>'
    elif re.match(_regex._expressions['url'], w):
        return '<URL>'
    else:
        w = ''.join(''.join(s)[:2] for _, s in itertools.groupby(w))
        return w

In [ ]:
word2idx = {'PAD': 0,'UNK':1, '_ROOT': 2}
tag2idx = {'PAD': 0, '_<ROOT>': 1}
char2idx = {'PAD': 0,'UNK':1, '_ROOT': 2}
word_idx = 3
tag_idx = 2
char_idx = 3

special_tokens = ['<NUM>', '<MONEY>', '<DATE>', '<URL>', '<EMAIL>']

for t in special_tokens:
    word2idx[t] = word_idx
    word_idx += 1
    char2idx[t] = char_idx
    char_idx += 1
    
word2idx, char2idx

In [ ]:
PAD = "_PAD"
PAD_POS = "_PAD_POS"
PAD_TYPE = "_<PAD>"
PAD_CHAR = "_PAD_CHAR"
ROOT = "_ROOT"
ROOT_POS = "_ROOT_POS"
ROOT_TYPE = "_<ROOT>"
ROOT_CHAR = "_ROOT_CHAR"
END = "_END"
END_POS = "_END_POS"
END_TYPE = "_<END>"
END_CHAR = "_END_CHAR"

def process_corpus(corpus, until = None):
    global word2idx, tag2idx, char2idx, word_idx, tag_idx, char_idx
    sentences, words, depends, labels, pos, chars = [], [], [], [], [], []
    temp_sentence, temp_word, temp_depend, temp_label, temp_pos = [], [], [], [], []
    first_time = True
    for sentence in corpus:
        try:
            if len(sentence):
                if sentence[0] == '#':
                    continue
                if first_time:
                    print(sentence)
                    first_time = False
                sentence = sentence.split('\t')
                for c in sentence[1]:
                    if c not in char2idx:
                        char2idx[c] = char_idx
                        char_idx += 1
                if sentence[7] not in tag2idx:
                    tag2idx[sentence[7]] = tag_idx
                    tag_idx += 1
                sentence[1] = preprocessing(sentence[1])
                if sentence[1] not in word2idx:
                    word2idx[sentence[1]] = word_idx
                    word_idx += 1
                temp_word.append(word2idx[sentence[1]])
                temp_depend.append(int(sentence[6]))
                temp_label.append(tag2idx[sentence[7]])
                temp_sentence.append(sentence[1])
                temp_pos.append(sentence[3])
            else:
                if len(temp_sentence) < 2 or len(temp_word) != len(temp_label):
                    temp_word = []
                    temp_depend = []
                    temp_label = []
                    temp_sentence = []
                    temp_pos = []
                    continue
                words.append([word2idx['_ROOT']] + temp_word)
                depends.append([0] + temp_depend)
                labels.append([tag2idx['_<ROOT>']] + temp_label)
                sentences.append([ROOT] + temp_sentence)
                pos.append([ROOT_POS] + temp_pos)
                char_ = [[char2idx['_ROOT']]]
                for w in temp_sentence:
                    if w in char2idx:
                        char_.append([char2idx[w]])
                    else:
                        char_.append([char2idx[c] for c in w])
                chars.append(char_)
                temp_word = []
                temp_depend = []
                temp_label = []
                temp_sentence = []
                temp_pos = []
        except Exception as e:
            print(e, sentence)
    return sentences[:-1], words[:-1], depends[:-1], labels[:-1], pos[:-1], chars[:-1]

In [ ]:
def _obtain_child_index_for_left2right(heads):
    child_ids = [[] for _ in range(len(heads))]
    # skip the symbolic root.
    for child in range(1, len(heads)):
        head = heads[child]
        child_ids[head].append(child)
    return child_ids


def _obtain_child_index_for_inside_out(heads):
    child_ids = [[] for _ in range(len(heads))]
    for head in range(len(heads)):
        # first find left children inside-out
        for child in reversed(range(1, head)):
            if heads[child] == head:
                child_ids[head].append(child)
        # second find right children inside-out
        for child in range(head + 1, len(heads)):
            if heads[child] == head:
                child_ids[head].append(child)
    return child_ids


def _obtain_child_index_for_depth(heads, reverse):
    def calc_depth(head):
        children = child_ids[head]
        max_depth = 0
        for child in children:
            depth = calc_depth(child)
            child_with_depth[head].append((child, depth))
            max_depth = max(max_depth, depth + 1)
        child_with_depth[head] = sorted(child_with_depth[head], key=lambda x: x[1], reverse=reverse)
        return max_depth

    child_ids = _obtain_child_index_for_left2right(heads)
    child_with_depth = [[] for _ in range(len(heads))]
    calc_depth(0)
    return [[child for child, depth in child_with_depth[head]] for head in range(len(heads))]


def _generate_stack_inputs(heads, types, prior_order):
    if prior_order == 'deep_first':
        child_ids = _obtain_child_index_for_depth(heads, True)
    elif prior_order == 'shallow_first':
        child_ids = _obtain_child_index_for_depth(heads, False)
    elif prior_order == 'left2right':
        child_ids = _obtain_child_index_for_left2right(heads)
    elif prior_order == 'inside_out':
        child_ids = _obtain_child_index_for_inside_out(heads)
    else:
        raise ValueError('Unknown prior order: %s' % prior_order)

    stacked_heads = []
    children = []
    siblings = []
    stacked_types = []
    skip_connect = []
    prev = [0 for _ in range(len(heads))]
    sibs = [0 for _ in range(len(heads))]
    stack = [0]
    position = 1
    while len(stack) > 0:
        head = stack[-1]
        stacked_heads.append(head)
        siblings.append(sibs[head])
        child_id = child_ids[head]
        skip_connect.append(prev[head])
        prev[head] = position
        if len(child_id) == 0:
            children.append(head)
            sibs[head] = 0
            stacked_types.append(tag2idx['PAD'])
            stack.pop()
        else:
            child = child_id.pop(0)
            children.append(child)
            sibs[head] = child
            stack.append(child)
            stacked_types.append(types[child])
        position += 1

    return stacked_heads, children, siblings, stacked_types, skip_connect

In [ ]:
_buckets = [10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100, 140]

def process_data(corpus, batch_size = 32):
    sentences, words, depends, labels, pos, chars = process_corpus(corpus)
    print(len(sentences), len(words), len(depends), len(labels), len(chars))

    data = [[] for _ in _buckets]
    max_char_length = [0 for _ in _buckets]

    for i in range(len(depends)):
        for bucket_id, bucket_size in enumerate(_buckets):
            if len(words[i]) < bucket_size:
                stacked_heads, children, siblings, stacked_types, skip_connect \
                = _generate_stack_inputs(depends[i], labels[i], 'deep_first')
                data[bucket_id].append([words[i], pos[i], depends[i], labels[i], chars[i],
                                        stacked_heads, children, siblings, stacked_types, skip_connect])
                max_len = max([len(char_seq) for char_seq in chars[i]])
                if max_char_length[bucket_id] < max_len:
                    max_char_length[bucket_id] = max_len
                break
    
    bucket_sizes = [len(data[b]) for b in range(len(_buckets))]
    X = []
    for bucket_id in range(len(_buckets)):
        bucket_length = _buckets[bucket_id]
        bucket_size = bucket_sizes[bucket_id]
        if not bucket_size:
            X.append([])
            continue
        char_length = max_char_length[bucket_id]

        wid_inputs = np.zeros([bucket_size, bucket_length], dtype=np.int32)
        cid_inputs = np.zeros([bucket_size, bucket_length, char_length], dtype=np.int32)
        hid_inputs = np.zeros([bucket_size, bucket_length], dtype=np.int32)
        tid_inputs = np.zeros([bucket_size, bucket_length], dtype=np.int32)

        stack_hid_inputs = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.int32)
        chid_inputs = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.int32)
        ssid_inputs = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.int32)
        stack_tid_inputs = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.int32)
        skip_connect_inputs = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.int32)

        masks_e = np.zeros([bucket_size, bucket_length], dtype=np.float32)
        masks_d = np.zeros([bucket_size, 2 * bucket_length - 1], dtype=np.float32)
        lengths_d = np.empty(bucket_size, dtype=np.int32)

        for i, inst in enumerate(data[bucket_id]):
            w, p, d, l, ch, stack_hids, chids, ssids, stack_tids, skip_ids = inst
            inst_size = len(w)
            wid_inputs[i, :inst_size] = w
            for c, cids in enumerate(ch):
                cid_inputs[i, c, :len(cids)] = cids
            tid_inputs[i, :inst_size] = l
            hid_inputs[i, :inst_size] = d
            masks_e[i, :inst_size] = 1.0

            inst_size_decoder = 2 * inst_size - 1
            stack_hid_inputs[i, :inst_size_decoder] = stack_hids
            chid_inputs[i, :inst_size_decoder] = chids
            ssid_inputs[i, :inst_size_decoder] = ssids
            stack_tid_inputs[i, :inst_size_decoder] = stack_tids
            skip_connect_inputs[i, :inst_size_decoder] = skip_ids
            masks_d[i, :inst_size_decoder] = 1.0

        x = (wid_inputs, cid_inputs, hid_inputs, tid_inputs, stack_hid_inputs, chid_inputs, ssid_inputs,
        stack_tid_inputs, masks_e, masks_d)
        X.append(x)

    train_X = []
    for X_ in X:
        if not len(X_):
            continue
        wid_inputs, cid_inputs, hid_inputs, tid_inputs, stack_hid_inputs, chid_inputs, ssid_inputs, stack_tid_inputs, masks_e, masks_d = X_
        for k in range(0, len(wid_inputs), batch_size):
            index = min(k + batch_size, len(wid_inputs))
            batch_w = wid_inputs[k: index]
            batch_c = cid_inputs[k: index]
            batch_heads = hid_inputs[k: index]
            batch_stacked_heads = stack_hid_inputs[k: index]
            batch_siblings = ssid_inputs[k: index]
            batch_children = chid_inputs[k: index]
            batch_stacked_types = stack_tid_inputs[k: index]
            batch_e = masks_e[k: index]
            batch_d = masks_d[k: index]
            batch_types = tid_inputs[k: index]
            batch_len = np.count_nonzero(batch_w, axis = 1)
            x = (batch_w, batch_c, batch_heads, batch_stacked_heads, batch_siblings, batch_children,
            batch_stacked_types, batch_e, batch_d, batch_types, batch_len)
            train_X.append(x)
            
    print('trainable batch size', len(train_X))
    return train_X

In [ ]:
with open('en_ewt-ud-dev.conllu') as fopen:
    dev = fopen.read().split('\n')

test = process_data(dev)

In [ ]:
with open('en_ewt-ud-test.conllu') as fopen:
    dev = fopen.read().split('\n')

test.extend(process_data(dev))

In [ ]:
with open('en_ewt-ud-train.conllu') as fopen:
    train = fopen.read().split('\n')

train = process_data(train)

In [ ]:
len(train), len(test)

In [ ]:
idx2word = {v:k for k, v in word2idx.items()}
idx2tag = {v:k for k, v in tag2idx.items()}
len(idx2word)

In [ ]:
import tensorflow as tf

In [ ]:
from enum import Enum

class PriorOrder(Enum):
    DEPTH = 0
    INSIDE_OUT = 1
    LEFT2RIGTH = 2

class BiAAttention:
    def __init__(self, input_size_encoder, input_size_decoder, num_labels):
        self.input_size_encoder = input_size_encoder
        self.input_size_decoder = input_size_decoder
        self.num_labels = num_labels
        
        self.W_d = tf.get_variable("W_d", shape=[self.num_labels, self.input_size_decoder],
           initializer=tf.contrib.layers.xavier_initializer())
        self.W_e = tf.get_variable("W_e", shape=[self.num_labels, self.input_size_encoder],
           initializer=tf.contrib.layers.xavier_initializer())
        self.U = tf.get_variable("U", shape=[self.num_labels, self.input_size_decoder, self.input_size_encoder],
           initializer=tf.contrib.layers.xavier_initializer())
        
    def forward(self, input_d, input_e, mask_d=None, mask_e=None):
        batch = tf.shape(input_d)[0]
        length_decoder = tf.shape(input_d)[1]
        length_encoder = tf.shape(input_e)[1]
        out_d = tf.expand_dims(tf.matmul(self.W_d, tf.transpose(input_d, [0, 2, 1])), 3)
        out_e = tf.expand_dims(tf.matmul(self.W_e, tf.transpose(input_e, [0, 2, 1])), 2)
        output = tf.matmul(tf.expand_dims(input_d, 1), self.U)
        output = tf.matmul(output, tf.transpose(tf.expand_dims(input_e, 1), [0, 1, 3, 2]))
        
        output = output + out_d + out_e
        
        if mask_d is not None:
            d = tf.expand_dims(tf.expand_dims(mask_d, 1), 3)
            e = tf.expand_dims(tf.expand_dims(mask_e, 1), 2)
            output = output * d * e
            
        return output
    
class BiLinear:
    def __init__(self, left_features, right_features, out_features):
        self.left_features = left_features
        self.right_features = right_features
        self.out_features = out_features
        
        self.U = tf.get_variable("U-bi", shape=[out_features, left_features, right_features],
           initializer=tf.contrib.layers.xavier_initializer())
        self.W_l = tf.get_variable("Wl", shape=[out_features, left_features],
           initializer=tf.contrib.layers.xavier_initializer())
        self.W_r = tf.get_variable("Wr", shape=[out_features, right_features],
           initializer=tf.contrib.layers.xavier_initializer())
    
    def forward(self, input_left, input_right):
        left_size = tf.shape(input_left)
        output_shape = tf.concat([left_size[:-1], [self.out_features]], axis = 0)
        batch = tf.cast(tf.reduce_prod(left_size[:-1]), tf.int32)
        input_left = tf.reshape(input_left, (batch, self.left_features))
        input_right = tf.reshape(input_right, (batch, self.right_features))
        tiled = tf.tile(tf.expand_dims(input_left, axis = 0), (self.out_features,1,1))
        output = tf.transpose(tf.reduce_sum(tf.matmul(tiled, self.U), axis = 2))
        output = output + tf.matmul(input_left, tf.transpose(self.W_l))\
        + tf.matmul(input_right, tf.transpose(self.W_r))
        
        return tf.reshape(output, output_shape)

class StackPointer:
    def __init__(self, word_dim, num_words, char_dim, num_chars, num_filters, kernel_size,
                 input_size_decoder, hidden_size, encoder_layers, decoder_layers,
                 num_labels, arc_space, type_space):
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size,
                                           initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.word_embedd = tf.Variable(tf.random_uniform([num_words, word_dim], -1, 1))
        self.char_embedd = tf.Variable(tf.random_uniform([num_chars, char_dim], -1, 1))
        self.conv1d = tf.layers.Conv1D(num_filters, kernel_size, 1, padding='VALID')
        self.num_labels = num_labels
        self.prior_order = PriorOrder.DEPTH
        self.encoder = tf.nn.rnn_cell.MultiRNNCell([cells(hidden_size) for _ in range(encoder_layers)])
        self.decoder = tf.nn.rnn_cell.MultiRNNCell([cells(hidden_size) for _ in range(decoder_layers)])
        
        self.src_dense = tf.layers.Dense(hidden_size)
        self.hx_dense = tf.layers.Dense(hidden_size)

        self.arc_h = tf.layers.Dense(arc_space)
        self.arc_c = tf.layers.Dense(arc_space)
        self.attention = BiAAttention(arc_space, arc_space, 1)

        self.type_h = tf.layers.Dense(type_space)
        self.type_c = tf.layers.Dense(type_space)
        self.bilinear = BiLinear(type_space, type_space, self.num_labels)
        
    def encode(self, input_word, input_char):
        word = tf.nn.embedding_lookup(self.word_embedd, input_word)
        char = tf.nn.embedding_lookup(self.char_embedd, input_char)
        b = tf.shape(char)[0]
        wl = tf.shape(char)[1]
        cl = tf.shape(char)[2]
        d = char.shape[3]
        char = tf.reshape(char, [b * wl, cl, d])
        char = tf.reduce_max(self.conv1d(char), axis = 1)
        char = tf.nn.tanh(char)
        d = char.shape[-1]
        char = tf.reshape(char, [b, wl, d])
        
        src_encoding = tf.concat([word, char], axis=2)
        output, hn = tf.nn.dynamic_rnn(self.encoder, src_encoding, dtype = tf.float32,
                                      scope = 'encoder')
        return output, hn
    
    def decode(self, output_encoder, heads, heads_stack, siblings, hn):
        batch = tf.shape(output_encoder)[0]
        batch_index = tf.range(0, batch)
        t = tf.transpose(heads_stack)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0), 
                                               tf.expand_dims(t, axis = 0)], axis = 0))
        src_encoding = tf.gather_nd(output_encoder, concatenated)
        
        mask_sibs = tf.expand_dims(tf.cast(tf.not_equal(siblings, 0), tf.float32), axis = 2)
        t = tf.transpose(siblings)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0), 
                                               tf.expand_dims(t, axis = 0)], axis = 0))
        output_enc_sibling = tf.gather_nd(output_encoder, concatenated) * mask_sibs
        src_encoding = src_encoding + output_enc_sibling
        
        t = tf.transpose(heads_stack)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0), 
                                       tf.expand_dims(t, axis = 0)],axis = 0))
        g = tf.transpose(tf.gather_nd(heads, concatenated))
        broadcasted = tf.broadcast_to(batch_index, tf.shape(g))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0), 
                                               tf.expand_dims(g, axis = 0)],axis = 0))
        output_enc_gpar = tf.gather_nd(output_encoder, concatenated)
        src_encoding = src_encoding + output_enc_gpar
        
        src_encoding = tf.nn.elu(self.src_dense(src_encoding))
        output, hn = tf.nn.dynamic_rnn(self.decoder, src_encoding, dtype = tf.float32,
                                      initial_state = hn,
                                      scope = 'decoder')
        return output, hn
    
    def loss(self, input_word, input_char, 
             heads, stacked_heads, children, siblings, stacked_types,
             mask_e, mask_d,
             label_smooth = 1.0):
        
        output_enc, hn_enc = self.encode(input_word, input_char)
        arc_c = tf.nn.elu(self.arc_c(output_enc))
        type_c = tf.nn.elu(self.type_c(output_enc))
        
        output_dec, _ = self.decode(output_enc, heads, stacked_heads, siblings, hn_enc)
        arc_h = tf.nn.elu(self.arc_h(output_dec))
        type_h = tf.nn.elu(self.type_h(output_dec))
        
        max_len_d = tf.shape(arc_h)[1]
        
        out_arc = tf.squeeze(self.attention.forward(arc_h, arc_c, mask_d=mask_d, mask_e=mask_e), axis = 1)
        batch = tf.shape(arc_c)[0]
        max_len_e = tf.shape(arc_c)[1]
        batch_index = tf.range(0, batch)
        
        t = tf.transpose(children)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0), 
                                               tf.expand_dims(t, axis = 0)], axis = 0))
        type_c = tf.gather_nd(type_c, concatenated)
        out_type = self.bilinear.forward(type_h, type_c)
        print(out_arc.shape,out_type.shape)
        
        minus_inf = -1e8
        minus_mask_d = (1 - mask_d) * minus_inf
        minus_mask_e = (1 - mask_e) * minus_inf
        
        out_arc = out_arc + tf.expand_dims(minus_mask_d, 2) + tf.expand_dims(minus_mask_e, 1)
        loss_arc = tf.nn.log_softmax(out_arc, axis = 2)
        loss_type = tf.nn.log_softmax(out_type, axis = 2)
        coverage = tf.cumsum(tf.exp(loss_arc), axis = 1)
        
        mask_leaf = tf.cast(tf.equal(children, stacked_heads), tf.float32)
        mask_non_leaf = (1.0 - mask_leaf)
        
        mask_d_2 = tf.expand_dims(mask_d, 2)
        mask_e_1 = tf.expand_dims(mask_e, 1)
        
        loss_arc = loss_arc * mask_d_2 * mask_e_1
        coverage = coverage * mask_d_2 * mask_e_1
        loss_type = loss_type * mask_d_2
        mask_leaf = mask_leaf * mask_d
        mask_non_leaf = mask_non_leaf * mask_d
        num_leaf = tf.reduce_sum(mask_leaf)
        num_non_leaf = tf.reduce_sum(mask_non_leaf)
        head_index = tf.tile(tf.expand_dims(tf.range(0, max_len_d), 1), [1, batch])
        
        t = tf.transpose(children)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0),
                                               tf.expand_dims(head_index, axis = 0),
                                               tf.expand_dims(t, axis = 0)], axis = 0))
        loss_arc = tf.gather_nd(loss_arc, concatenated)
        
        t = tf.transpose(stacked_types)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0),
                                               tf.expand_dims(head_index, axis = 0),
                                               tf.expand_dims(t, axis = 0)], axis = 0))
        loss_type = tf.gather_nd(loss_type, concatenated)
        
        loss_arc_leaf = loss_arc * mask_leaf
        loss_arc_non_leaf = loss_arc * mask_non_leaf

        loss_type_leaf = loss_type * mask_leaf
        loss_type_non_leaf = loss_type * mask_non_leaf
        
        loss_cov = tf.clip_by_value(coverage - 2.0, 0.0, 100.0)
        
        return (tf.reduce_sum(-loss_arc_leaf) / num_leaf, 
                tf.reduce_sum(-loss_arc_non_leaf) / num_non_leaf,
                tf.reduce_sum(-loss_type_leaf) / num_leaf, 
                tf.reduce_sum(-loss_type_non_leaf) / num_non_leaf,
                tf.reduce_sum(loss_cov) / (num_leaf + num_non_leaf), 
                num_leaf, 
                num_non_leaf)
    
class Model:
    def __init__(self, learning_rate = 1e-3, cov = 0.0):
        self.stackpointer = StackPointer(word_dim = 128, 
                            num_words = len(word2idx), 
                            char_dim = 128, 
                            num_chars = len(char2idx), 
                            num_filters = 128, 
                            kernel_size = 3,
                            input_size_decoder = 256, 
                            hidden_size = 256, 
                            encoder_layers = 1, 
                            decoder_layers = 1,
                            num_labels = len(tag2idx), 
                            arc_space = 128, 
                            type_space = 128)
        self.words = tf.placeholder(tf.int32, (None, None))
        self.chars = tf.placeholder(tf.int32, (None, None, None))
        self.heads = tf.placeholder(tf.int32, (None, None))
        self.stacked_heads = tf.placeholder(tf.int32, (None, None))
        self.siblings = tf.placeholder(tf.int32, (None, None))
        self.childrens = tf.placeholder(tf.int32, (None, None))
        self.stacked_types = tf.placeholder(tf.int32, (None, None))
        self.mask_e = tf.placeholder(tf.float32, (None, None))
        self.mask_d = tf.placeholder(tf.float32, (None, None))
        loss_arc_leaf, loss_arc_non_leaf, \
        loss_type_leaf, loss_type_non_leaf, \
        loss_cov, num_leaf, num_non_leaf = self.stackpointer.loss(self.words, self.chars, self.heads, 
                               self.stacked_heads, self.childrens, 
                               self.siblings, self.stacked_types,
                               self.mask_e, self.mask_d)
        loss_arc = loss_arc_leaf + loss_arc_non_leaf
        loss_type = loss_type_leaf + loss_type_non_leaf
        self.cost = loss_arc + loss_type + cov * loss_cov
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())

In [ ]:
prior_order = model.stackpointer.prior_order

def decode_sentence(output_enc, arc_c, type_c, hx, beam, length, ordered, leading_symbolic):
    def valid_hyp(base_id, child_id, head):
        if constraints[base_id, child_id]:
            return False
        elif not ordered or prior_order == PriorOrder.DEPTH or child_orders[base_id, head] == 0:
            return True
        elif self.prior_order == PriorOrder.LEFT2RIGTH:
            return child_id > child_orders[base_id, head]
        else:
            if child_id < head:
                return child_id < child_orders[base_id, head] < head
            else:
                return child_id > child_orders[base_id, head]
        
    length = output_enc.shape[0] if length is None else length
    hx = tuple([hx])
            
    stacked_heads = [[0] for _ in range(beam)]
    grand_parents = [[0] for _ in range(beam)]
    siblings = [[0] for _ in range(beam)]
    children = np.zeros((beam, 2 * length - 1))
    stacked_types = np.zeros((beam, 2 * length - 1))
    
    children = np.zeros((beam, 2 * length - 1))
    stacked_types = np.zeros((beam, 2 * length - 1))
    hypothesis_scores = [0]
    constraints = np.zeros([beam, length], dtype=np.bool)
    constraints[:, 0] = True
    child_orders = np.zeros([beam, length], dtype=np.int64)

    new_stacked_heads = [[] for _ in range(beam)]
    new_grand_parents = [[] for _ in range(beam)]
    new_siblings = [[] for _ in range(beam)]
    new_skip_connects = [[] for _ in range(beam)]
    new_children = np.zeros((beam, 2 * length - 1))
    new_stacked_types = np.zeros((beam, 2 * length - 1))
    num_hyp = 1
    num_step = 2 * length - 1
    for t in range(num_step):
        heads = np.array([stacked_heads[i][-1] for i in range(num_hyp)])
        gpars = np.array([grand_parents[i][-1] for i in range(num_hyp)])
        sibs = np.array([siblings[i].pop() for i in range(num_hyp)])
        src_encoding = output_enc[heads]
        mask_sibs = np.expand_dims((np.array(sibs) != 0).astype(np.float32), axis = 1)
        output_enc_sibling = output_enc[sibs] * mask_sibs
        src_encoding = src_encoding + output_enc_sibling
        output_enc_gpar = output_enc[gpars]
        src_encoding = src_encoding + output_enc_gpar
        src_encoding = tf.nn.elu(model.stackpointer.src_dense(src_encoding))
        output_dec, hx = model.stackpointer.decoder(src_encoding, hx)
        arc_h = tf.nn.elu(model.stackpointer.arc_h(tf.expand_dims(output_dec, axis = 1)))
        type_h = tf.nn.elu(model.stackpointer.type_h(output_dec))
        out_arc = model.stackpointer.attention.forward(arc_h, tf.expand_dims(arc_c, 0))
        out_arc = tf.squeeze(tf.squeeze(out_arc, axis = 1), axis = 1)
        hyp_scores, type_h = sess.run([tf.nn.log_softmax(out_arc, axis = 1), type_h])
        new_hypothesis_scores = np.expand_dims(hypothesis_scores[:num_hyp], axis = 1) + hyp_scores
        new_hypothesis_scores = new_hypothesis_scores.reshape((-1))
        hyp_index = np.argsort(new_hypothesis_scores)[::-1]
        new_hypothesis_scores = np.sort(new_hypothesis_scores)[::-1]
        base_index = (hyp_index // length)
        child_index = hyp_index % length
        cc = 0
        ids = []
        new_constraints = np.zeros([beam, length], dtype=np.bool)
        new_child_orders = np.zeros([beam, length], dtype=np.int64)
        for id_ in range(num_hyp * length):
            base_id = base_index[id_]
            if base_id:
                ids.append(id_)
                continue
            child_id = child_index[id_]
            head = heads[base_id]
            new_hyp_score = new_hypothesis_scores[id_]
            if child_id == head:
                if head != 0 or t + 1 == num_step:
                    new_constraints[cc] = constraints[base_id]
                    new_child_orders[cc] = child_orders[base_id]

                    new_stacked_heads[cc] = [stacked_heads[base_id][i] for i in range(len(stacked_heads[base_id]))]
                    new_stacked_heads[cc].pop()

                    new_grand_parents[cc] = [grand_parents[base_id][i] for i in range(len(grand_parents[base_id]))]
                    new_grand_parents[cc].pop()

                    new_siblings[cc] = [siblings[base_id][i] for i in range(len(siblings[base_id]))]

                    new_children[cc] = children[base_id]
                    new_children[cc, t] = child_id

                    hypothesis_scores[cc] = new_hyp_score
                    ids.append(id_)
                    cc += 1
            elif valid_hyp(base_id, child_id, head):
                new_constraints[cc] = constraints[base_id]
                new_constraints[cc, child_id] = True

                new_child_orders[cc] = child_orders[base_id]
                new_child_orders[cc, head] = child_id

                new_stacked_heads[cc] = [stacked_heads[base_id][i] for i in range(len(stacked_heads[base_id]))]
                new_stacked_heads[cc].append(child_id)

                new_grand_parents[cc] = [grand_parents[base_id][i] for i in range(len(grand_parents[base_id]))]
                new_grand_parents[cc].append(head)

                new_siblings[cc] = [siblings[base_id][i] for i in range(len(siblings[base_id]))]
                new_siblings[cc].append(child_id)
                new_siblings[cc].append(0)

                new_children[cc] = children[base_id]
                new_children[cc, t] = child_id

                hypothesis_scores[cc] = new_hyp_score
                ids.append(id_)
                cc += 1
                    
            if cc == beam:
                break
            
        num_hyp = len(ids)
        if num_hyp == 0:
            return None
        else:
            index = np.array(ids)
        base_index = base_index[index]
        child_index = child_index[index]
        out_type = model.stackpointer.bilinear.forward(type_h[base_index], type_c[child_index])
        hyp_type_scores = sess.run(tf.nn.log_softmax(out_type, axis = 1))
        hyp_types = np.argmax(hyp_type_scores, axis = 1)
        hyp_type_scores = np.max(hyp_type_scores, axis = 1)
        hypothesis_scores[:num_hyp] = hypothesis_scores[:num_hyp] + hyp_type_scores

        for i in range(num_hyp):
            base_id = base_index[i]
            new_stacked_types[i] = stacked_types[base_id]
            new_stacked_types[i, t] = hyp_types[i]

        stacked_heads = [[new_stacked_heads[i][j] for j in range(len(new_stacked_heads[i]))] for i in range(num_hyp)]
        grand_parents = [[new_grand_parents[i][j] for j in range(len(new_grand_parents[i]))] for i in range(num_hyp)]
        siblings = [[new_siblings[i][j] for j in range(len(new_siblings[i]))] for i in range(num_hyp)]
        constraints = new_constraints
        child_orders = new_child_orders
        children = np.copy(new_children)
        stacked_types = np.copy(new_stacked_types)
        
    children = children[0].astype(np.int32)
    stacked_types = stacked_types[0].astype(np.int32)
    heads = np.zeros(length, dtype=np.int32)
    types = np.zeros(length, dtype=np.int32)
    stack = [0]
    for i in range(num_step):
        head = stack[-1]
        child = children[i]
        type_ = stacked_types[i]
        if child != head:
            heads[child] = head
            types[child] = type_
            stack.append(child)
        else:
            stacked_types[i] = 0
            stack.pop()

    return heads, types, length, children, stacked_types   
        
def decode(input_word, input_char, length = None, beam = 1, leading_symbolic=0, ordered=True):
    output, hn = model.stackpointer.encode(input_word, input_char)
    arc_c, type_c, output, hn = sess.run([tf.nn.elu(model.stackpointer.arc_c(output)), 
                              tf.nn.elu(model.stackpointer.type_c(output)),
                              output, hn])
    batch, max_len_e, _ = output.shape

    heads = np.zeros([batch, max_len_e], dtype=np.int32)
    types = np.zeros([batch, max_len_e], dtype=np.int32)

    children = np.zeros([batch, 2 * max_len_e - 1], dtype=np.int32)
    stack_types = np.zeros([batch, 2 * max_len_e - 1], dtype=np.int32)
    
    for b in range(batch):
        sent_len = None if length is None else length[b]
        state = tf.nn.rnn_cell.LSTMStateTuple(hn[0].c[b:b+1], hn[0].h[b:b+1])
        preds = decode_sentence(output[b], arc_c[b], type_c[b], state, beam, sent_len, ordered, leading_symbolic)
        if preds is None:
            preds = decode_sentence(output[b], arc_c[b], type_c[b], state, beam, 
                                         sent_len, False, leading_symbolic)
        hids, tids, sent_len, chids, stids = preds
        heads[b, :sent_len] = hids
        types[b, :sent_len] = tids

        children[b, :2 * sent_len - 1] = chids
        stack_types[b, :2 * sent_len - 1] = stids

    return heads, types, children, stack_types

In [ ]:
# batch_w = wid_inputs[:2]
# batch_c = cid_inputs[:2]
# batch_heads = hid_inputs[:2]
# batch_stacked_heads = stack_hid_inputs[:2]
# batch_siblings = ssid_inputs[:2]
# batch_children = chid_inputs[:2]
# batch_stacked_types = stack_tid_inputs[:2]
# batch_e = masks_e[:2]
# batch_d = masks_d[:2]
# batch_types = tid_inputs[:2]
# batch_len = np.count_nonzero(batch_w, axis = 1)

# NUM_SYMBOLIC_TAGS = 3
# heads_pred, types_pred, _, _ = decode(batch_w, batch_c, leading_symbolic = NUM_SYMBOLIC_TAGS)
# evaluate(batch_w, heads_pred, types_pred, batch_heads, batch_types, batch_len)

In [ ]:
def evaluate(words, heads_pred, types_pred, heads, types, lengths,
             symbolic_root=False, symbolic_end=False):
    batch_size, _ = words.shape
    ucorr = 0.
    lcorr = 0.
    total = 0.
    ucomplete_match = 0.
    lcomplete_match = 0.

    corr_root = 0.
    total_root = 0.
    start = 1 if symbolic_root else 0
    end = 1 if symbolic_end else 0
    for i in range(batch_size):
        ucm = 1.
        lcm = 1.
        for j in range(start, lengths[i] - end):

            total += 1
            if heads[i, j] == heads_pred[i, j]:
                ucorr += 1
                if types[i, j] == types_pred[i, j]:
                    lcorr += 1
                else:
                    lcm = 0
            else:
                ucm = 0
                lcm = 0

            if heads[i, j] == 0:
                total_root += 1
                corr_root += 1 if heads_pred[i, j] == 0 else 0

        ucomplete_match += ucm
        lcomplete_match += lcm

    return (ucorr, lcorr, total, ucomplete_match, lcomplete_match), \
           (corr_root, total_root), batch_size

In [ ]:
from tqdm import tqdm
epoch = 20

for e in range(epoch):
    train_loss, test_loss = [], []
    pbar = tqdm(range(len(train)), desc = 'train minibatch loop')
    for k in pbar:
        batch_w, batch_c, batch_heads, batch_stacked_heads, batch_siblings, batch_children, \
        batch_stacked_types, batch_e, batch_d, batch_types, batch_len = train[k]
        feed_dict = {model.words: batch_w,
                     model.chars: batch_c,
                     model.heads: batch_heads,
                     model.stacked_heads: batch_stacked_heads,
                     model.childrens: batch_children,
                     model.siblings: batch_siblings,
                     model.stacked_types: batch_stacked_types,
                     model.mask_e: batch_e,
                     model.mask_d: batch_d}
        cost, _ = sess.run([model.cost, model.optimizer], feed_dict = feed_dict)
        train_loss.append(cost)
        pbar.set_postfix(cost = cost)
    
    pbar = tqdm(range(len(test)), desc = 'test minibatch loop')
    for k in pbar:
        batch_w, batch_c, batch_heads, batch_stacked_heads, batch_siblings, batch_children, \
        batch_stacked_types, batch_e, batch_d, batch_types, batch_len = test[k]
        feed_dict = {model.words: batch_w,
                     model.chars: batch_c,
                     model.heads: batch_heads,
                     model.stacked_heads: batch_stacked_heads,
                     model.childrens: batch_children,
                     model.siblings: batch_siblings,
                     model.stacked_types: batch_stacked_types,
                     model.mask_e: batch_e,
                     model.mask_d: batch_d}
        cost = sess.run(model.cost, feed_dict = feed_dict)
        test_loss.append(cost)
        pbar.set_postfix(cost = cost)
    
    print('epoch: %d, avg train loss: %f, avg test loss: %f'%(e + 1, np.mean(train_loss),
                                                             np.mean(test_loss)))